# Usar tabular env

In [1]:
# Bibliotecas
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer
import matplotlib.colors as mcolors
import folium
from folium import plugins
from shapely.geometry import Polygon
import os
import random
import pprint
from pprint import pprint
import csv
import json
import random
from matplotlib.colors import LogNorm
import seaborn as sns
import plotnine
from plotnine import *
from shapely.geometry import mapping

In [2]:
# 1. Recriar a paleta tim.colors() do R com mais tons
def tim_colors(n=1000):
    """Recria a paleta tim.colors() do R com mais detalhes"""
    colors = [
        '#000080',  # azul marinho
        '#0000CC',  # azul escuro
        '#0040FF',  # azul
        '#0080FF',  # azul claro
        '#00BFFF',  # azul céu
        '#00FFFF',  # ciano
        '#00FFBF',  # ciano-verde
        '#00FF80',  # verde-ciano
        '#00FF40',  # verde claro
        '#00FF00',  # verde
        '#40FF00',  # verde-amarelo
        '#80FF00',  # amarelo-verde
        '#BFFF00',  # amarelo claro
        '#FFFF00',  # amarelo
        '#FFBF00',  # amarelo-laranja
        '#FF8000',  # laranja
        '#FF4000',  # laranja-vermelho
        '#FF0000'   # vermelho
    ]
    
    cmap = mcolors.LinearSegmentedColormap.from_list("tim", colors, N=n)
    return [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, n)]

# 2. Gerar a paleta
pal14 = tim_colors(1000)

# 3. Parâmetros do grid
grid_spacing = 109.45  # metros
half_spacing = grid_spacing / 2  # 54.725 metros

In [3]:
pasta = '/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/mediana'
arquivos_csv = [
    'predicoes_unit_Sarandi.csv', 
    'predicoes_unit_Rolandia.csv',
    'predicoes_unit_Maringa.csv',
    'predicoes_unit_Marialva.csv',
    'predicoes_unit_Mandaguari.csv',
    'predicoes_unit_Londrina.csv',
    'predicoes_unit_Jandaia.csv',
    'predicoes_unit_Ibipora.csv',
    'predicoes_unit_Cambira.csv',
    'predicoes_unit_Cambe.csv',
    'predicoes_unit_Arapongas.csv',
    'predicoes_unit_Apucarana.csv'
]

dataframes = []
for arquivo in arquivos_csv:
    df = pd.read_csv(os.path.join(pasta, arquivo))
    df['cidade'] = arquivo.replace('predicoes_', '').replace('.csv', '')
    dataframes.append(df)

df_consolidado = pd.concat(dataframes, ignore_index=True)
df_consolidado.to_csv(os.path.join(pasta, 'predicoes_consolidadas.csv'), index=False)

In [4]:
# 4. Criar quadrados em UTM para cada ponto
def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

# 5. Criar geometrias UTM para cada ponto
utm_polygons = []
for idx, row in df_consolidado.iterrows():
    square_utm = create_square_utm(row['utm_x'], row['utm_y'], half_spacing)
    utm_polygons.append(square_utm)

print(f"Criados {len(utm_polygons)} quadrados em UTM")

# 6. Converter geometrias UTM para lat/lon
transformer = Transformer.from_crs("EPSG:32722", "EPSG:4326", always_xy=True)

def transform_polygon_to_latlon(polygon_utm):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# Converter todos os polígonos
latlon_polygons = []
for polygon_utm in utm_polygons:
    latlon_polygon = transform_polygon_to_latlon(polygon_utm)
    latlon_polygons.append(latlon_polygon)

print(f"Convertidos {len(latlon_polygons)} quadrados para lat/lon")

# 7. Obter min e max dos valores preditos (em log)
min_log_price = df_consolidado['price_predicted'].min()
max_log_price = df_consolidado['price_predicted'].max()
price_range = max_log_price - min_log_price

# Criar breaks baseados no range dos dados
breaks = np.linspace(min_log_price, max_log_price, 18)  

# Converter log para valores reais
real_prices = np.exp(breaks)

# Formatação das labels
labs = [f"R$ {price:,.0f}" for price in real_prices]

def get_color_from_price(price):
    """Obtém cor da paleta baseada no preço"""
    normalized = (price - min_log_price) / price_range
    color_index = int(normalized * (len(pal14) - 1))
    color_index = max(0, min(color_index, len(pal14) - 1))  # Garantir índice válido
    return pal14[color_index]

# 8. Criar mapa base COM SATÉLITE (CORRIGIDO)
center_utm_x = df_consolidado['utm_x'].mean()
center_utm_y = df_consolidado['utm_y'].mean()
center_lon, center_lat = transformer.transform(center_utm_x, center_utm_y)

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)


# 9. Criar GeoJSON FeatureCollection
print("Criando GeoJSON otimizado...")

features = []
for idx in range(len(latlon_polygons)):
    polygon_coords = latlon_polygons[idx]
    price = df_consolidado.iloc[idx]['price_predicted']
    color = get_color_from_price(price)
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
            "popup": f"Predicted Median Unit Value for Urban Land (2024): R$ {np.exp(price):,.0f}/m²"
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

print(f"✅ GeoJSON criado com {len(features)} features")

# Adicionar ao mapa usando GeoJson
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(m)

print("✅ GeoJSON adicionado ao mapa!")



# 10. Criar legenda HTML
legend_html = f"""
<div style="position: fixed; 
           top: 10px; right: 10px; width: 280px; height: 570px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:12px; padding: 10px; overflow-y: scroll;">
<p><b>Predicted Median Unit Value for Urban Land (R$/m²: 2024)</b></p>
<table style="font-size:10px; border-collapse: collapse;">
"""

# Adicionar faixas de cores na legenda
for i in range(len(breaks)-1):
    color_index = int((i / (len(breaks)-2)) * (len(pal14)-1))
    color = pal14[color_index]
    
    # Calcular preços reais para as faixas
    price_min = np.exp(breaks[i])
    price_max = np.exp(breaks[i+1])
    
    legend_html += f"""
    <tr>
        <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
        <td style="padding-left:8px; vertical-align:middle;">R$ {price_min:,.0f}/m² - R$ {price_max:,.0f}/m²</td>
    </tr>
    """

legend_html += f"""
</table>
<div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
<p style="margin:0; font-weight:bold;">Paradigm:</p>
<ul style="margin:5px 0; padding-left:15px;">
    <li>Spatial Resolution: {grid_spacing:.1f}m × {grid_spacing:.1f}m</li>
    <li>Urban vacant parcel with paradigm area of 363m²</li>
</ul>
<p style="margin:10px 0 0 0; font-style:italic; font-size:9px;">
</p>
</div>
</div>
"""


m.get_root().html.add_child(folium.Element(legend_html))


# 11. Salvar mapa
m.save('/Users/fjcosta/Documents/landCoverlandValue/landvalue/median_LandValueMapTabularPFN_NPr.html')


Criados 73399 quadrados em UTM
Convertidos 73399 quadrados para lat/lon
Criando GeoJSON otimizado...
✅ GeoJSON criado com 73399 features
✅ GeoJSON adicionado ao mapa!


In [5]:
pasta = '/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/IC_80'

arquivos_csv = [
    'lim_sup_predicoes_unit_Sarandi.csv', 
    'lim_sup_predicoes_unit_Rolandia.csv',
    'lim_sup_predicoes_unit_Maringa.csv',
    'lim_sup_predicoes_unit_Marialva.csv',
    'lim_sup_predicoes_unit_Mandaguari.csv',
    'lim_sup_predicoes_unit_Londrina.csv',
    'lim_sup_predicoes_unit_Jandaia.csv',
    'lim_sup_predicoes_unit_Ibipora.csv',
    'lim_sup_predicoes_unit_Cambira.csv',
    'lim_sup_predicoes_unit_Cambe.csv',
    'lim_sup_predicoes_unit_Arapongas.csv',
    'lim_sup_predicoes_unit_Apucarana.csv'
]

dataframes = []
for arquivo in arquivos_csv:
    df = pd.read_csv(os.path.join(pasta, arquivo))
    df['cidade'] = arquivo.replace('lim_sup_predicoes_', '').replace('.csv', '')
    dataframes.append(df)

df_consolidado_lim_sup = pd.concat(dataframes, ignore_index=True)
df_consolidado_lim_sup.to_csv(os.path.join(pasta, 'lim_sup_predicoes_consolidadas.csv'), index=False)

In [ ]:
# 4. Criar quadrados em UTM para cada ponto
def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

# 5. Criar geometrias UTM para cada ponto
utm_polygons = []
for idx, row in df_consolidado_lim_sup.iterrows():
    square_utm = create_square_utm(row['utm_x'], row['utm_y'], half_spacing)
    utm_polygons.append(square_utm)

print(f"Criados {len(utm_polygons)} quadrados em UTM")

# 6. Converter geometrias UTM para lat/lon
transformer = Transformer.from_crs("EPSG:32722", "EPSG:4326", always_xy=True)

def transform_polygon_to_latlon(polygon_utm):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# Converter todos os polígonos
latlon_polygons = []
for polygon_utm in utm_polygons:
    latlon_polygon = transform_polygon_to_latlon(polygon_utm)
    latlon_polygons.append(latlon_polygon)

print(f"Convertidos {len(latlon_polygons)} quadrados para lat/lon")

# 7. Obter min e max dos valores preditos (em log)
min_log_price = df_consolidado_lim_sup['price_predicted'].min()
max_log_price = df_consolidado_lim_sup['price_predicted'].max()
price_range = max_log_price - min_log_price

# Criar breaks baseados no range dos dados
breaks = np.linspace(min_log_price, max_log_price, 18)  

# Converter log para valores reais
real_prices = np.exp(breaks)

# Formatação das labels
labs = [f"R$ {price:,.0f}" for price in real_prices]

def get_color_from_price(price):
    """Obtém cor da paleta baseada no preço"""
    normalized = (price - min_log_price) / price_range
    color_index = int(normalized * (len(pal14) - 1))
    color_index = max(0, min(color_index, len(pal14) - 1))  # Garantir índice válido
    return pal14[color_index]

# 8. Criar mapa base COM SATÉLITE (CORRIGIDO)
center_utm_x = df_consolidado_lim_sup['utm_x'].mean()
center_utm_y = df_consolidado_lim_sup['utm_y'].mean()
center_lon, center_lat = transformer.transform(center_utm_x, center_utm_y)

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)


# 9. Criar GeoJSON FeatureCollection
print("Criando GeoJSON otimizado...")

features = []
for idx in range(len(latlon_polygons)):
    polygon_coords = latlon_polygons[idx]
    price = df_consolidado_lim_sup.iloc[idx]['price_predicted']
    color = get_color_from_price(price)
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
            "popup": f"Predicted Upper Bound Unit Value for Urban Land (2024)<br>Confidence Level=0.80: R$ {np.exp(price):,.0f}/m²"
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

print(f"✅ GeoJSON criado com {len(features)} features")

# Adicionar ao mapa usando GeoJson
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(m)

print("✅ GeoJSON adicionado ao mapa!")



# 10. Criar legenda HTML
legend_html = f"""
<div style="position: fixed; 
           top: 10px; right: 10px; width: 280px; height: 570px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:12px; padding: 10px; overflow-y: scroll;">
<p><b>Predicted Upper Bound Unit Value for Urban Land (R$/m²: 2024)<br>
Confidence Level=0.80</b></p>
<table style="font-size:10px; border-collapse: collapse;">
"""

# Adicionar faixas de cores na legenda
for i in range(len(breaks)-1):
    color_index = int((i / (len(breaks)-2)) * (len(pal14)-1))
    color = pal14[color_index]
    
    # Calcular preços reais para as faixas
    price_min = np.exp(breaks[i])
    price_max = np.exp(breaks[i+1])
    
    legend_html += f"""
    <tr>
        <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
        <td style="padding-left:8px; vertical-align:middle;">R$ {price_min:,.0f}/m² - R$ {price_max:,.0f}/m²</td>
    </tr>
    """

legend_html += f"""
</table>
<div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
<p style="margin:0; font-weight:bold;">Paradigm:</p>
<ul style="margin:5px 0; padding-left:15px;">
    <li>Spatial Resolution: {grid_spacing:.1f}m × {grid_spacing:.1f}m</li>
    <li>Urban vacant parcel with paradigm area of 363m²</li>
</ul>
<p style="margin:10px 0 0 0; font-style:italic; font-size:9px;">
</p>
</div>
</div>
"""


m.get_root().html.add_child(folium.Element(legend_html))


# 11. Salvar mapa
m.save('/Users/fjcosta/Documents/landCoverlandValue/landvalue/upperBound_LandValueMapTabularPFN_NPr.html')


In [ ]:
pasta = '/Users/fjcosta/Documents/projects/Tabular/resultados_inferencia/IC_80'

arquivos_csv = [
    'lim_inf_predicoes_unit_Sarandi.csv', 
    'lim_inf_predicoes_unit_Rolandia.csv',
    'lim_inf_predicoes_unit_Maringa.csv',
    'lim_inf_predicoes_unit_Marialva.csv',
    'lim_inf_predicoes_unit_Mandaguari.csv',
    'lim_inf_predicoes_unit_Londrina.csv',
    'lim_inf_predicoes_unit_Jandaia.csv',
    'lim_inf_predicoes_unit_Ibipora.csv',
    'lim_inf_predicoes_unit_Cambira.csv',
    'lim_inf_predicoes_unit_Cambe.csv',
    'lim_inf_predicoes_unit_Arapongas.csv',
    'lim_inf_predicoes_unit_Apucarana.csv'
]

dataframes = []
for arquivo in arquivos_csv:
    df = pd.read_csv(os.path.join(pasta, arquivo))
    df['cidade'] = arquivo.replace('lim_inf_predicoes_', '').replace('.csv', '')
    dataframes.append(df)

df_consolidado_lim_inf = pd.concat(dataframes, ignore_index=True)
df_consolidado_lim_inf.to_csv(os.path.join(pasta, 'lim_inf_predicoes_consolidadas.csv'), index=False)

In [ ]:
# 4. Criar quadrados em UTM para cada ponto
def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

# 5. Criar geometrias UTM para cada ponto
utm_polygons = []
for idx, row in df_consolidado_lim_inf.iterrows():
    square_utm = create_square_utm(row['utm_x'], row['utm_y'], half_spacing)
    utm_polygons.append(square_utm)

print(f"Criados {len(utm_polygons)} quadrados em UTM")

# 6. Converter geometrias UTM para lat/lon
transformer = Transformer.from_crs("EPSG:32722", "EPSG:4326", always_xy=True)

def transform_polygon_to_latlon(polygon_utm):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# Converter todos os polígonos
latlon_polygons = []
for polygon_utm in utm_polygons:
    latlon_polygon = transform_polygon_to_latlon(polygon_utm)
    latlon_polygons.append(latlon_polygon)

print(f"Convertidos {len(latlon_polygons)} quadrados para lat/lon")

# 7. Obter min e max dos valores preditos (em log)
min_log_price = df_consolidado_lim_inf['price_predicted'].min()
max_log_price = df_consolidado_lim_inf['price_predicted'].max()
price_range = max_log_price - min_log_price

# Criar breaks baseados no range dos dados
breaks = np.linspace(min_log_price, max_log_price, 18)  

# Converter log para valores reais
real_prices = np.exp(breaks)

# Formatação das labels
labs = [f"R$ {price:,.0f}" for price in real_prices]

def get_color_from_price(price):
    """Obtém cor da paleta baseada no preço"""
    normalized = (price - min_log_price) / price_range
    color_index = int(normalized * (len(pal14) - 1))
    color_index = max(0, min(color_index, len(pal14) - 1))  # Garantir índice válido
    return pal14[color_index]

# 8. Criar mapa base COM SATÉLITE (CORRIGIDO)
center_utm_x = df_consolidado_lim_inf['utm_x'].mean()
center_utm_y = df_consolidado_lim_inf['utm_y'].mean()
center_lon, center_lat = transformer.transform(center_utm_x, center_utm_y)

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)


# 9. Criar GeoJSON FeatureCollection
print("Criando GeoJSON otimizado...")

features = []
for idx in range(len(latlon_polygons)):
    polygon_coords = latlon_polygons[idx]
    price = df_consolidado_lim_inf.iloc[idx]['price_predicted']
    color = get_color_from_price(price)
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.7,
            "popup": f"Predicted Lower Bound Unit Value for Urban Land (2024)<br>Confidence Level=0.80: R$ {np.exp(price):,.0f}/m²"
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

print(f"✅ GeoJSON criado com {len(features)} features")

# Adicionar ao mapa usando GeoJson
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(m)

print("✅ GeoJSON adicionado ao mapa!")



# 10. Criar legenda HTML
legend_html = f"""
<div style="position: fixed; 
           top: 10px; right: 10px; width: 280px; height: 570px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:12px; padding: 10px; overflow-y: scroll;">
<p><b>Predicted Lower Bound Unit Value for Urban Land (R$/m²: 2024)<br>
Confidence Level=0.80</b></p>
<table style="font-size:10px; border-collapse: collapse;">
"""

# Adicionar faixas de cores na legenda
for i in range(len(breaks)-1):
    color_index = int((i / (len(breaks)-2)) * (len(pal14)-1))
    color = pal14[color_index]
    
    # Calcular preços reais para as faixas
    price_min = np.exp(breaks[i])
    price_max = np.exp(breaks[i+1])
    
    legend_html += f"""
    <tr>
        <td style="background-color:{color}; width:20px; height:10px; border:1px solid #ccc;"></td>
        <td style="padding-left:8px; vertical-align:middle;">R$ {price_min:,.0f}/m² - R$ {price_max:,.0f}/m²</td>
    </tr>
    """

legend_html += f"""
</table>
<div style="font-size:10px; margin-top:15px; color:#666; line-height:1.4;">
<p style="margin:0; font-weight:bold;">Paradigm:</p>
<ul style="margin:5px 0; padding-left:15px;">
    <li>Spatial Resolution: {grid_spacing:.1f}m × {grid_spacing:.1f}m</li>
    <li>Urban vacant parcel with paradigm area of 363m²</li>
</ul>
<p style="margin:10px 0 0 0; font-style:italic; font-size:9px;">
</p>
</div>
</div>
"""


m.get_root().html.add_child(folium.Element(legend_html))


# 11. Salvar mapa
m.save('/Users/fjcosta/Documents/landCoverlandValue/landvalue/lowerBound_LandValueMapTabularPFN_NPr.html')

Criados 73399 quadrados em UTM
Convertidos 73399 quadrados para lat/lon
Criando GeoJSON otimizado...
✅ GeoJSON criado com 73399 features
✅ GeoJSON adicionado ao mapa!
